In [ ]:
import yfinance as yf
import pandas as pd
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta

In [100]:

# Download data function
def download_data(ticker_symbols, start_date,  end_date):
        data = {}
        for symbol in ticker_symbols:
                symbol_data = yf.download(symbol, start=start_date, end=end_date)
                data[symbol] = symbol_data['Adj Close']
        symbol_prices = pd.DataFrame(data)
        return symbol_prices


In [101]:
# returns annualized return percentage
def calculate_annualized_returns(ticker_data):
    daily_returns = ticker_data.pct_change().dropna()
    annualized_returns = daily_returns.mean() * 252  # 252 trading days in a year
    return annualized_returns

In [102]:
# returns covariance matrix
def calculate_annualized_covariance(ticker_data):
    daily_returns = ticker_data.pct_change().dropna()
    annualized_covariance = daily_returns.cov() * 252
    return annualized_covariance

In [104]:
# returns a portfolio's returns and standard deviation given a specific weight
def portfolio_performance(weights, returns, covariance_matrix):
    # return = sum of weights*returns
    portfolio_return = np.sum(returns * weights)
    # standard deviations = sqrt(variance) = sqrt(weights.T * covariance * weights) 
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    return portfolio_return, portfolio_std_dev



In [105]:
# function that optimizese a portfolio given a target return percent
def optimize_portfolio(returns, covariance_matrix, target_return):
    num_tickers = len(returns)
    args = (returns, covariance_matrix)
    constraints = [{# Weights sum to 1
                    'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   # Return matches target return
                   {'type': 'eq', 'fun': lambda x: portfolio_performance(x, returns, covariance_matrix)[0] - target_return}]  
    bounds = tuple((0, 1) for i in range(num_tickers))  # Weight bounds (0 to 1)

    # Initial guess for equal weights
    initial_weights = num_tickers * [1. / num_tickers]
    def vol_function(weights):
          return portfolio_performance(weights, returns, covariance_matrix)[1]

    result = minimize(vol_function, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [108]:
def main():
        #pre-defined list of etfs
        etfs = ['SPY', 'VOO', 'SCHD', 'VTI', 'QQQ']

        # calculate the today's date and the date exactly 10 years in the past
        present_date = date.today()
        past_date = present_date - relativedelta(years=10)
        present_date = present_date.strftime('%Y-%m-%d')
        past_date = past_date.strftime('%Y-%m-%d')

        # download the etf data from dates above
        etf_data = download_data(etfs, past_date, present_date)
        prices = pd.DataFrame(etf_data)

        # Calculate annualized returns and covariance
        annualized_returns = calculate_annualized_returns(prices)
        annualized_covariance = calculate_annualized_covariance(prices)

        
        # Ask user for target return percentage (e.g., 5% annual return = 0.05)
        target_return = float(input("Enter your target annual return percentage (as a decimal, e.g., 0.05 for 5%): "))

        # Optimize the portfolio for minimum volatility at the target return
        optimized_portfolio = optimize_portfolio(annualized_returns, annualized_covariance, target_return)

        # Display the optimized portfolio weights
        optimized_weights = optimized_portfolio.x
        print("Optimized Portfolio Weights (Min Volatility):")
        for etf, weight in zip(etfs, optimized_weights):
            print(f"{etf}: {weight:.4f}")

        # Calculate and display the sum of the optimized weights
        weights_sum = np.sum(optimized_weights)
        print(f"Sum of Optimized Weights: {weights_sum:.4f}")

        # Equal weights for comparison
        equal_weights = np.array([1 / len(etfs)] * len(etfs))  # Equal weights for all ETFs

        # Performance of the optimized portfolio
        opt_return, opt_std_dev = portfolio_performance(optimized_weights, annualized_returns, annualized_covariance)

        # Performance of the equally weighted portfolio
        eq_return, eq_std_dev = portfolio_performance(equal_weights, annualized_returns, annualized_covariance)

        # Print the comparison between optimized and equal weights portfolios
        print("\nComparison of Optimized vs Equal Weights Portfolio:")
        print(f"Optimized Portfolio - Return: {opt_return:.4f}, Volatility: {opt_std_dev:.4f}")
        print(f"Equal Weights Portfolio - Return: {eq_return:.4f}, Volatility: {eq_std_dev:.4f}")


     
        
    
# risk = weights*covariance matrix * weights.T
# returns = returns vector * weights.T or by dot product
# min risk and max
# point on the curve of efficiency w = lambda * (covariance^-1) * return vectors

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Optimized Portfolio Weights (Min Volatility):
SPY: 0.0000
VOO: 0.0000
SCHD: 0.6948
VTI: 0.0000
QQQ: 0.3052
Sum of Optimized Weights: 1.0000

Comparison of Optimized vs Equal Weights Portfolio:
Optimized Portfolio - Return: 0.1394, Volatility: 0.1714
Equal Weights Portfolio - Return: 0.1394, Volatility: 0.1781
